NOTE: Plotting requires plotly to be installed
ie., `poetry add plotly=">=4.8,<=6"`

In [ ]:
from pathlib import Path
from teehr import Evaluation, Configuration
from teehr.models.pydantic_table_models import (
    Attribute
)
import tempfile
import geopandas as gpd
import numpy as np


TEST_DATA_DIR = Path(Path.cwd().parents[1], "tests", "data", "v0_3_test_study")
GEOJSON_GAGES_FILEPATH = Path(TEST_DATA_DIR, "geo", "gages.geojson")
PRIMARY_TIMESERIES_FILEPATH = Path(
    TEST_DATA_DIR, "timeseries", "test_short_obs.parquet"
)
CROSSWALK_FILEPATH = Path(TEST_DATA_DIR, "geo", "crosswalk.csv")
SECONDARY_TIMESERIES_FILEPATH = Path(
    TEST_DATA_DIR, "timeseries", "test_short_fcast.parquet"
)
GEO_FILEPATH = Path(TEST_DATA_DIR, "geo")

In [ ]:
tmpdir = tempfile.TemporaryDirectory(prefix="teehr-")

In [ ]:
ev = Evaluation(dir_path=Path(tmpdir.name))

# Enable logging
ev.enable_logging()

# Clone the template
ev.clone_template()

# Load the location data
ev.locations.load_spatial(in_path=GEOJSON_GAGES_FILEPATH)

ev.configurations.add(
    Configuration(
        name="usgs_observations",
        type="primary",
        description="test primary configuration"
    )
)

# Load the timeseries data and map over the fields and set constants
ev.primary_timeseries.load_parquet(
    in_path=PRIMARY_TIMESERIES_FILEPATH,
    field_mapping={
        "reference_time": "reference_time",
        "value_time": "value_time",
        "configuration": "configuration_name",
        "measurement_unit": "unit_name",
        "variable_name": "variable_name",
        "value": "value",
        "location_id": "location_id"
    },
    constant_field_values={
        "unit_name": "m^3/s",
        "variable_name": "streamflow_hourly_inst",
        "configuration_name": "usgs_observations"
    }
)

In [ ]:
sdf = ev.primary_timeseries.to_sdf()

In [ ]:
sdf.show()

In [ ]:
sdf = ev.primary_timeseries.to_sdf()
(
    sdf
    .orderBy("value_time")
    .plot
    .line(
        x="value_time",
        y="value",
        title="Primary Timeseries Data",
        line_group="location_id",
        color="location_id",
    )
)
